In [1]:
#describe each location with companies in side
import pandas as pd
import numpy as np
import os
import pygeohash as pgh
from math import *
from sklearn.metrics.pairwise import euclidean_distances
import random
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import precision_recall_curve, roc_curve, auc
from matplotlib import pylab
from sklearn.preprocessing import normalize
pjoin = os.path.join

In [17]:
#function_base
def getPosNegdat(dat):
    """
    dat: pos pair of data (location,company,geo,distance)
    return pos/neg pair of data, same structure of dat except one more column for label
    """
    shuffle_dat = dat.sample(frac=1).reset_index(drop=True)

    # shuffle_dat.head()

    twin_dat = dat.join(shuffle_dat,how='left',lsuffix='_left',rsuffix='_right')
    twin_dat = twin_dat[twin_dat['atlas_location_uuid_left'] != twin_dat['atlas_location_uuid_right']]
    print(len(twin_dat))
    twin_dat.head()

    neg_datA = twin_dat[['duns_number_left','atlas_location_uuid_right','longitude_loc_right','latitude_loc_right']]
    neg_datA = neg_datA.rename(columns={'duns_number_left':'duns_number','atlas_location_uuid_right':'atlas_location_uuid','longitude_loc_right':'longitude_loc','latitude_loc_right':'latitude_loc'})

    neg_datB = twin_dat[['duns_number_right','atlas_location_uuid_left','longitude_loc_left','latitude_loc_left']]
    neg_datB = neg_datB.rename(columns={'duns_number_right':'duns_number','atlas_location_uuid_left':'atlas_location_uuid','longitude_loc_left':'longitude_loc','latitude_loc_left':'latitude_loc'})

    neg_dat = pd.concat([neg_datA,neg_datB],axis=0)
    neg_dat['label'] = 0
    dat['label'] = 1
    res_dat = pd.concat([dat[['duns_number','atlas_location_uuid','longitude_loc','latitude_loc','label']],neg_dat],axis=0)
    print('Neg dat num:',len(neg_dat),';Pos dat num:',len(dat))
    return res_dat

def splitdat(dat,key_column=['duns_number'],right_colunm='atlas_location_uuid_tr',rate_tr=0.8):
    """
    split the <company,location> pair into training/testing dat
    """
    tr = dat.sample(frac=rate_tr)
    tt = pd.merge(dat,tr,on=key_column,how='left',suffixes=['','_tr'])
    tt = tt[tt[right_colunm].isnull()]
    tt = tt[list(tr.columns)]
    print('Train dat:', len(tr), 'Test dat:', len(tt))
    return tr,tt

#data process
def onehotdat(dat,key_column:list,dummy_na=True):
    dat[key_column] = dat[key_column].astype(str)
    dum_dat = pd.get_dummies(dat[key_column],dummy_na=dummy_na)#it has nan itself
    return dum_dat

def split2num(emp_range:str):
    max_emp_val = emp_range.replace(' ','').split('-')
    if len(max_emp_val)<2:
        return 10
    else:
        return float(max_emp_val[1])
    
def max_col(dat,col,minval=1):
    dat[col] = dat[col].apply(lambda r:max(r,minval))

def comp_dat_process(dat):
    """
    pd -> company key,cont_feature,spec_feature,dum_feature
    """
    one_hot_col_name = ['major_industry_category','location_type','primary_sic_2_digit']
    spec_col_name = 'emp_here_range'
    cont_col_name = ['emp_here','emp_total','sales_volume_us','square_footage']

    print('doing one-hot...')
    dum_dat = onehotdat(dat,one_hot_col_name)
    
    print('extract continuous...')
    cont_dat = dat[cont_col_name].fillna(value=0).astype(float)
    
    print('specific feature')
    spec_dat = dat[spec_col_name].fillna(value='1-10').astype(str)
    spec_dat = spec_dat.apply(lambda row: split2num(row))
    
    max_col(cont_dat,'emp_here',1)
    
    res_dat = dat[['duns_number']].join([cont_dat,spec_dat,dum_dat],how='left')
    assert(len(res_dat)==len(dum_dat))
    assert(len(res_dat)==len(cont_dat))
    assert(len(res_dat)==len(spec_dat))
    return res_dat

def location_dat_process(dat):
    """
    pd -> location key,cont_feature,dum_feature
    """
    one_hot_col_name = ['building_class']
    cont_col_name = ['score_predicted_eo','score_employer','num_emp_weworkcore','num_poi_weworkcore',
                     'pct_wwcore_employee','pct_wwcore_business','num_retail_stores','num_doctor_offices',
                     'num_eating_places','num_drinking_places','num_hotels','num_fitness_gyms',
                     'population_density','pct_female_population','median_age','income_per_capita',
                     'pct_masters_degree','walk_score','bike_score']

    print('doing one-hot...')
    dum_dat = onehotdat(dat,one_hot_col_name,False)
    print(len(dum_dat))
    
    print('extract continuous...')
    cont_dat = dat[cont_col_name].fillna(value=0).astype(float)
    print(len(cont_dat))
    
    res_dat = dat[['atlas_location_uuid']].join([cont_dat,dum_dat],how='left')
    print(len(res_dat))
    assert(len(res_dat)==len(dum_dat))
    assert(len(res_dat)==len(cont_dat))
    return {'data':res_dat,
            'cont_feat_num':len(list(cont_dat.columns)),
            'dum_feat_num':len(list(dum_dat.columns))}

def comp_transpd2np(featdat,trdat,ttdat,not_col_name):
    tr_feat = pd.merge(trdat,featdat,on='duns_number',how='inner')
#     print(col_list)
    col_list = [ n for n in list(tr_feat.columns) if n not in not_col_name ] 
    trainX = tr_feat.loc[:,col_list].to_numpy()
    trainY = tr_feat[['atlas_location_uuid','longitude_loc','latitude_loc']].to_numpy()
    
    tt_feat = pd.merge(ttdat,featdat,on='duns_number',how='inner')
    col_list = [ n for n in list(tt_feat.columns) if n not in not_col_name ] 
#     print(col_list)
    testX = tt_feat.loc[:,col_list].to_numpy()
    testY = tt_feat[['atlas_location_uuid','longitude_loc','latitude_loc']].to_numpy()
    return trainX,trainY,testX,testY

def transpd2np(featdatC,featdatL,pairdat,cont_col_nameC,cont_col_nameL,not_feat_col):
    tr_feat = pd.merge(pairdat,featdatC,on='duns_number',how='inner')
    XCC = tr_feat.loc[:,cont_col_nameC].to_numpy()
    out_col = []
    out_col.extend(not_feat_col)
    out_col.extend(cont_col_nameC)
    dum_col_nameC = [col for col in list(tr_feat.columns) if col not in out_col]
    XDC = tr_feat.loc[:,dum_col_nameC].to_numpy()

    tr_feat = pd.merge(pairdat,featdatL,on='atlas_location_uuid',how='inner')
    XCL = tr_feat.loc[:,cont_col_nameL].to_numpy()
    out_col = []
    out_col.extend(not_feat_col)
    out_col.extend(cont_col_nameL)
    dum_col_nameL = [col for col in list(tr_feat.columns) if col not in out_col]
    XDL = tr_feat.loc[:,dum_col_nameL].to_numpy()

    Y = pairdat[['label']].to_numpy()
    return XCC,XDC,XCL,XDL,Y

def transpd2np_train_test(featdatC,featdatL,trdat,ttdat):
    not_feat_col = ['duns_number',
                     'atlas_location_uuid',
                     'longitude_loc',
                     'latitude_loc',
                     'label']
    cont_col_nameC = ['emp_here','emp_total','sales_volume_us','square_footage','emp_here_range']
    cont_col_nameL = ['score_predicted_eo','score_employer','num_emp_weworkcore','num_poi_weworkcore',
                     'pct_wwcore_employee','pct_wwcore_business','num_retail_stores','num_doctor_offices',
                     'num_eating_places','num_drinking_places','num_hotels','num_fitness_gyms',
                     'population_density','pct_female_population','median_age','income_per_capita',
                     'pct_masters_degree','walk_score','bike_score']
    trXCC,trXDC,trXCL,trXDL,trY = transpd2np(featdatC,featdatL,trdat,cont_col_nameC,cont_col_nameL,not_feat_col)
    ttXCC,ttXDC,ttXCL,ttXDL,ttY = transpd2np(featdatC,featdatL,ttdat,cont_col_nameC,cont_col_nameL,not_feat_col)
    
    trXC = np.concatenate([trXCC,trXCL],axis=1)
    trXD = np.concatenate([trXDC,trXDL],axis=1)
    ttXC = np.concatenate([ttXCC,ttXCL],axis=1)
    ttXD = np.concatenate([ttXDC,ttXDL],axis=1)
#     trXC = 1.0*trXCC
#     trXD = 1.0*trXDC
#     ttXC = 1.0*ttXCC
#     ttXD = 1.0*ttXDC
    del trXCC,trXDC,trXCL,trXDL,ttXCC,ttXDC,ttXCL,ttXDL
    return trXC,trXD,ttXC,ttXD,trY,ttY

def transpdfeat_w_pair(featdat,pairdat,key_col,not_col_name):
    tr_feat = pd.merge(pairdat,featdat,on=key_col,how='inner').fillna(0)
    feat_col_name = [col for col in list(tr_feat.columns) if col not in not_col_name]
    X = tr_feat.loc[:,feat_col_name].to_numpy()
    return X

def normalize_dat_v2(trX,ttX,axis=0):
    center = trX.mean(axis=axis)
    center = np.expand_dims(center,axis)
    scale = trX.std(axis=axis)
    scale = np.expand_dims(scale,axis)
    
    trX = (trX-center)/scale
    ttX = (ttX-center)/scale
    return trX,ttX

def get_para_normalize_dat(trX,axis=0):
    center = trX.mean(axis=axis)
    scale = trX.std(axis=axis)
    scale += 1e-4
    return center,scale

def apply_para_normalize_dat(X,center,scale,axis=0):
    center = np.expand_dims(center,axis)
    scale = np.expand_dims(scale,axis)
    X = (X-center)/scale
    return X

def normalize_dat(trX,ttX,cols=5,axis=0):
    D = trX[:,:cols]
    center = D.mean(axis=axis)
    center = np.expand_dims(center,axis)
    scale = D.std(axis=axis)
    scale = np.expand_dims(scale,axis)
    
    trX[:,:cols] = (D-center)/scale
    ttX[:,:cols] = (ttX[:,:cols]-center)/scale
    
def calc_topk_acc_v2(QRscore,y_truth_cat,R_cat,k=3):
    """
    QRscore: similarity score matrix shape [Q,R]
    y_truth: index(related with R) of truth label of Query
    """
    y_truth_cat = y_truth_cat.reshape(-1,1)
    max_k_preds = QRscore.argsort(axis=1)[:, -k:][:, ::-1] #得到top-k max label
    max_k_cat = R_cat[max_k_preds]
    match_array = np.logical_or.reduce(max_k_cat==y_truth_cat, axis=1) #得到匹配结果
    topk_acc_score = match_array.sum() / match_array.shape[0]
    return topk_acc_score

def calc_topk_acc_cat_all(QRscore,y_truth_cat,R_cat,k=3):
    """
    QRscore: similarity score matrix shape [Q,R]
    y_truth: index(related with R) of truth label of Query
    return top1-topk acc
    """
    res = []
    y_truth_cat = y_truth_cat.reshape(-1,1)
    max_k_preds = QRscore.argsort(axis=1)[:, -k:][:, ::-1] #得到top-k max label
    max_k_cat = R_cat[max_k_preds]
    M = max_k_cat==y_truth_cat
    for k in range(M.shape[1]):
        match_array = np.logical_or.reduce(M[:,:k+1], axis=1) #得到匹配结果
        topk_acc_score = match_array.sum() / match_array.shape[0]
        res.append(topk_acc_score)
    return res

def transpd2np_single(featdatC,featdatL,trdat):
    not_feat_col = ['duns_number',
                     'atlas_location_uuid',
                     'longitude_loc',
                     'latitude_loc',
                     'label']
    cont_col_nameC = ['emp_here','emp_total','sales_volume_us','square_footage','emp_here_range']
    cont_col_nameL = ['score_predicted_eo','score_employer','num_emp_weworkcore','num_poi_weworkcore',
                     'pct_wwcore_employee','pct_wwcore_business','num_retail_stores','num_doctor_offices',
                     'num_eating_places','num_drinking_places','num_hotels','num_fitness_gyms',
                     'population_density','pct_female_population','median_age','income_per_capita',
                     'pct_masters_degree','walk_score','bike_score']
    trXCC,trXDC,trXCL,trXDL,trY = transpd2np(featdatC,featdatL,trdat,cont_col_nameC,cont_col_nameL,not_feat_col)
    
    trXC = np.concatenate([trXCC,trXCL],axis=1)
    trXD = np.concatenate([trXDC,trXDL],axis=1)
    del trXCC,trXDC,trXCL,trXDL
    return trXC,trXD,trY

def onehot2cat(x):
    """
    x: each row is a sample
    """
    return [np.where(r==1)[0][0] for r in x]

def get_loc_feat_by_comp(proc_comp_dat,pair_dat):
    tr_feat = pd.merge(pair_dat[['atlas_location_uuid','duns_number']],proc_comp_dat,on='duns_number',how='inner')
#     tr_feat = tr_feat.fillna(0)
    tr_feat = tr_feat.groupby(['atlas_location_uuid']).mean().drop(columns=['duns_number'])
    return tr_feat


In [24]:
##Multi training data generator(multi city)
#如果不合并所有数据在进行dummy 会出现一些category在某些城市不出现的情况，从而导致问题
#只进行正负样本的划分!!!!!

def transpd2np_single(featdat,cont_col_name:list,not_feat_col:list,id_col_name:list):
    XC = featdat.loc[:,cont_col_name].to_numpy()
    out_col = not_feat_col+cont_col_name
    dum_col_name = [col for col in list(featdat.columns) if col not in out_col]
    XD = featdat.loc[:,dum_col_name].to_numpy()
    Y = featdat[id_col_name].to_numpy()
    return XC,XD,Y,cont_col_name,dum_col_name,id_col_name

def splitdat(dat,key_column=['duns_number'],right_colunm='atlas_location_uuid_tr',rate_tr=0.8):
    """
    split the <company,location> pair into training/testing dat
    """
    tr = dat.sample(frac=rate_tr)
    tt = pd.merge(dat,tr,on=key_column,how='left',suffixes=['','_tr'])
    tt = tt[tt[right_colunm].isnull()]
    tt = tt[list(tr.columns)]
    print('Train dat:', len(tr), 'Test dat:', len(tt))
    return tr,tt

datapath = '/Users/yefeichen/Database/location_recommender_system/'
cfile = ['dnb_pa.csv','dnb_sf.csv','dnb_sj.csv']
lfile = 'location_scorecard_190912.csv'
clfile = ['PA.csv','SF.csv','SJ.csv']

not_feat_col = ['duns_number',
                 'atlas_location_uuid',
                 'longitude_loc',
                 'latitude_loc',
                 'label']
cont_col_nameC = ['emp_here','emp_total','sales_volume_us','square_footage','emp_here_range']
cont_col_nameL = ['score_predicted_eo','score_employer','num_emp_weworkcore','num_poi_weworkcore',
                 'pct_wwcore_employee','pct_wwcore_business','num_retail_stores','num_doctor_offices',
                 'num_eating_places','num_drinking_places','num_hotels','num_fitness_gyms',
                 'population_density','pct_female_population','median_age','income_per_capita',
                 'pct_masters_degree','walk_score','bike_score']
key_col_comp = ['duns_number']
key_col_loc = ['atlas_location_uuid']

ind_city = 0

train_test_val_pairs = []
dat_comp_pds = []
dat_loc_pds = []

pdlls = [] #all location feat pd list
pdccs = []
for ind_city in range(3):
    pdc = pd.read_csv(pjoin(datapath,cfile[ind_city]))
    pdl = pd.read_csv(pjoin(datapath,lfile))
    pdcl = pd.read_csv(pjoin(datapath,clfile[ind_city]))
    
    print('generating train_val_test csv')
    #train_test_val_pairs :[ duns_number, atlas_location_uuid, label, city, fold ]
    pdcl['label'] = 1
    tr,tt = splitdat(pdcl)
    
    #training pair ==> pair format with positive only
    train_pos_pair = tr.reset_index(drop=True)[['duns_number','atlas_location_uuid','label']]
    #testing pair ==> pair format with positive and negative both
    testing_pos_pair = tt.reset_index(drop=True)[['duns_number','atlas_location_uuid','label']]
    

    all_loc_name = pdcl[['atlas_location_uuid']].groupby(['atlas_location_uuid'])[['atlas_location_uuid']].first().reset_index(drop=True)
    all_loc_name['key'] = 0
    testing_pos_pair['key'] = 0
    
    testing_pair = pd.merge(testing_pos_pair,all_loc_name,on='key',how='left',suffixes=['_left','_right']).reset_index(drop=True)
    
    testing_pair = testing_pair.rename(columns={'atlas_location_uuid_left':'groundtruth','atlas_location_uuid_right':'atlas_location_uuid'})
    testing_pair = testing_pair[['duns_number','atlas_location_uuid','groundtruth']]
    
    train_pos_pair['fold'] = 0
    testing_pair['fold'] = 2

    train_test_val_pair = pd.concat([train_pos_pair,testing_pair],sort=False)
    train_test_val_pair['city'] = ind_city
    train_test_val_pairs.append(train_test_val_pair)
    print(len(train_test_val_pair))
    print('train_val_test_location_company Done')
#for loop end


print('Final merge...')
train_test_val_pair = pd.concat(train_test_val_pairs)

train_test_val_pair.to_csv('train_val_test_location_company_all.csv')
print('All Done')

generating train_val_test csv
Train dat: 4975 Test dat: 1244
442863
train_val_test_location_company Done
generating train_val_test csv
Train dat: 45192 Test dat: 11298
16303014
train_val_test_location_company Done
generating train_val_test csv
Train dat: 20032 Test dat: 5008
3094944
train_val_test_location_company Done
Final merge...
All Done


In [25]:
train_test_val_pair[train_test_val_pair['fold']==2].head()

,duns_number,atlas_location_uuid,label,fold,groundtruth,city
0,2127360.0,003fd16c-0dee-24a1-1adf-0b47d6120f3d,NaN,2,7a687b49-1055-24f6-ae84-e390438fe3c4,0
1,2127360.0,0060ef47-b121-60b9-1983-02ebdb144dcf,NaN,2,7a687b49-1055-24f6-ae84-e390438fe3c4,0
2,2127360.0,0084ae4b-c24c-0795-d1e6-a4f58444d39b,NaN,2,7a687b49-1055-24f6-ae84-e390438fe3c4,0
3,2127360.0,00a68dc6-f955-c581-9a54-64404bb0ef6a,NaN,2,7a687b49-1055-24f6-ae84-e390438fe3c4,0
4,2127360.0,01109439-6b76-0fc9-2d96-cfdb79708a80,NaN,2,7a687b49-1055-24f6-ae84-e390438fe3c4,0


In [23]:
train_test_val_pair.head()

,duns_number,atlas_location_uuid,label,fold,groundtruth,city
0,29479548.0,6a801297-10fb-599b-be2e-b360b6895768,1.0,0,NaN,0
1,24135820.0,a79c730c-7d89-85c9-fd40-b5c927e7297d,1.0,0,NaN,0
2,81122493.0,20125fd9-b2d4-3e34-0a35-fb0278da235d,1.0,0,NaN,0
3,116848312.0,2517756c-5a9b-e6ac-007f-e9bb7fb92611,1.0,0,NaN,0
4,75624198.0,be388ce1-c781-98ae-dd20-5d6e44eb292d,1.0,0,NaN,0


In [29]:
##Multi training data generator(multi city)
#如果不合并所有数据在进行dummy 会出现一些category在某些城市不出现的情况，从而导致问题

def transpd2np_single(featdat,cont_col_name:list,not_feat_col:list,id_col_name:list):
    XC = featdat.loc[:,cont_col_name].to_numpy()
    out_col = not_feat_col+cont_col_name
    dum_col_name = [col for col in list(featdat.columns) if col not in out_col]
    XD = featdat.loc[:,dum_col_name].to_numpy()
    Y = featdat[id_col_name].to_numpy()
    return XC,XD,Y,cont_col_name,dum_col_name,id_col_name

datapath = '/Users/yefeichen/Database/location_recommender_system/'
cfile = ['dnb_pa.csv','dnb_sf.csv','dnb_sj.csv']
lfile = 'location_scorecard_190912.csv'
clfile = ['PA.csv','SF.csv','SJ.csv']

not_feat_col = ['duns_number',
                 'atlas_location_uuid',
                 'longitude_loc',
                 'latitude_loc',
                 'label']
cont_col_nameC = ['emp_here','emp_total','sales_volume_us','square_footage','emp_here_range']
cont_col_nameL = ['score_predicted_eo','score_employer','num_emp_weworkcore','num_poi_weworkcore',
                 'pct_wwcore_employee','pct_wwcore_business','num_retail_stores','num_doctor_offices',
                 'num_eating_places','num_drinking_places','num_hotels','num_fitness_gyms',
                 'population_density','pct_female_population','median_age','income_per_capita',
                 'pct_masters_degree','walk_score','bike_score']
key_col_comp = ['duns_number']
key_col_loc = ['atlas_location_uuid']

ind_city = 0

train_test_val_pairs = []
dat_comp_pds = []
dat_loc_pds = []

pdlls = [] #all location feat pd list
pdccs = []
for ind_city in range(3):
    pdc = pd.read_csv(pjoin(datapath,cfile[ind_city]))
    pdl = pd.read_csv(pjoin(datapath,lfile))
    pdcl = pd.read_csv(pjoin(datapath,clfile[ind_city]))
    
    print('generating train_val_test csv')
    #train_test_val_pairs :[ duns_number, atlas_location_uuid, label, city, fold ]
    pair_dat = getPosNegdat(pdcl)
    tr,tt = splitdat(pair_dat,key_column=['duns_number','atlas_location_uuid'],right_colunm='label_tr',rate_tr=0.8)
    #training pair ==> pair format with positive only
    train_pos_pair = tr[tr['label']==1].groupby(['duns_number','atlas_location_uuid','label']).first().reset_index()[['duns_number','atlas_location_uuid','label']]
    #testing pair ==> pair format with positive and negative both
    testing_pair = tt.reset_index()[['duns_number','atlas_location_uuid','label']]
    
    train_pos_pair['fold'] = 0
    testing_pair['fold'] = 2

    train_test_val_pair = pd.concat([train_pos_pair,testing_pair])
    train_test_val_pair['city'] = ind_city
    train_test_val_pairs.append(train_test_val_pair)
    print(len(train_test_val_pair))
    print('train_val_test_location_company Done')
    
    #building features
    col_list = list(pdl.columns)
    pdll = pdl.merge(pdcl,how='inner',on=['atlas_location_uuid'],suffixes=['','_right'])
    pdll = pdll[pdll['duns_number'].isnull()==False]
    pdll = pdll.groupby(['atlas_location_uuid']).first().reset_index()
    pdll = pdll[col_list]
    pdlls.append(pdll)
    
    #company feature
    pdccs.append(pdc)
    
#for loop end
pdlls = pd.concat(pdlls,axis=0)
pdccs = pd.concat(pdccs,axis=0)

    
#building feature
pdlls = pdlls.reset_index()
proc_pdl = location_dat_process(pdlls)

#company feature
pdccs = pdccs.reset_index()
proc_pdc = comp_dat_process(pdccs)
print(len(proc_pdc))
    

print('start saving company and location feature...')

XC_comp,XD_comp,Y_comp,c_comp_name,d_comp_name,y_comp_name = transpd2np_single(proc_pdc,cont_col_nameC,not_feat_col,id_col_name=key_col_comp)
XC_loc,XD_loc,Y_loc,c_loc_name,d_loc_name,y_loc_name = transpd2np_single(proc_pdl['data'],cont_col_nameL,not_feat_col,id_col_name=key_col_loc)

C_comp,S_comp = get_para_normalize_dat(XC_comp)
C_loc,S_loc = get_para_normalize_dat(XC_loc)
XC_comp = apply_para_normalize_dat(XC_comp,C_comp,S_comp)
XC_loc = apply_para_normalize_dat(XC_loc,C_loc,S_loc)
    
X_comp = np.concatenate([Y_comp,XC_comp,XD_comp],axis=1)
X_loc = np.concatenate([Y_loc,XC_loc,XD_loc],axis=1)

dat_comp_pd = pd.DataFrame(data=X_comp,columns=y_comp_name+c_comp_name+d_comp_name)
dat_loc_pd = pd.DataFrame(data=X_loc,columns=y_loc_name+c_loc_name+d_loc_name)
    
print(dat_comp_pd.to_numpy().mean())
print(dat_loc_pd.to_numpy()[:,1:].mean())
print(dat_comp_pd.shape)

print('Done')

print('Final merge...')
train_test_val_pair = pd.concat(train_test_val_pairs)

train_test_val_pair.to_csv('train_val_test_location_company_all.csv')
dat_comp_pd.to_csv('company_feat.csv')
dat_loc_pd.to_csv('location_feat.csv')
print('All Done')

generating train_val_test csv
6168
Neg dat num: 12336 ;Pos dat num: 6219
Train dat: 14844 Test dat: 3701
8628
train_val_test_location_company Done
generating train_val_test csv
56372
Neg dat num: 112744 ;Pos dat num: 56490
Train dat: 135387 Test dat: 33801
79139
train_val_test_location_company Done
generating train_val_test csv
24964
Neg dat num: 49928 ;Pos dat num: 25040
Train dat: 59974 Test dat: 14973
35071
train_val_test_location_company Done
doing one-hot...
2405
extract continuous...
2405
2405
doing one-hot...
extract continuous...
specific feature
123764
start saving company and location feature...
2126648.004115018
0.04347826086956533
(123764, 103)
Done
Final merge...
All Done


In [36]:
dat_comp_pd.to_numpy().mean(axis=0)
dat_loc_pd.to_numpy()[:,1:].mean(axis=0)

array([-2.3450864719733038e-17, -2.1531690729296026e-15,
       -2.0367168335327197e-16, -1.7666625842995733e-16,
       -2.8750390841436485e-16, 8.849470013332329e-17,
       3.1390921278382806e-18, 2.0284074720178536e-16,
       -3.3066642517214124e-16, 1.2493125037601086e-15,
       3.154787588477472e-16, 1.2983838998173453e-15,
       -1.5464645041556234e-16, 1.3539642957278936e-16,
       -9.564998366001349e-17, 2.2061354821969326e-16,
       1.5418481922029202e-17, 1.0366851752185921e-15,
       1.3594115438320836e-15, 0.15051975051975053, 0.5455301455301456,
       0.22952182952182953, 0.07442827442827443], dtype=object)

In [70]:
##Multi training data generator(multi city)

def transpd2np_single(featdat,cont_col_name:list,not_feat_col:list,id_col_name:list):
    XC = featdat.loc[:,cont_col_name].to_numpy()
    out_col = not_feat_col+cont_col_name
    dum_col_name = [col for col in list(featdat.columns) if col not in out_col]
    XD = featdat.loc[:,dum_col_name].to_numpy()
    Y = featdat[id_col_name].to_numpy()
    return XC,XD,Y,cont_col_name,dum_col_name,id_col_name

datapath = '/Users/yefeichen/Database/location_recommender_system/'
cfile = ['dnb_pa.csv','dnb_sf.csv','dnb_sj.csv']
lfile = 'location_scorecard_190912.csv'
clfile = ['PA.csv','SF.csv','SJ.csv']

not_feat_col = ['duns_number',
                 'atlas_location_uuid',
                 'longitude_loc',
                 'latitude_loc',
                 'label']
cont_col_nameC = ['emp_here','emp_total','sales_volume_us','square_footage','emp_here_range']
cont_col_nameL = ['score_predicted_eo','score_employer','num_emp_weworkcore','num_poi_weworkcore',
                 'pct_wwcore_employee','pct_wwcore_business','num_retail_stores','num_doctor_offices',
                 'num_eating_places','num_drinking_places','num_hotels','num_fitness_gyms',
                 'population_density','pct_female_population','median_age','income_per_capita',
                 'pct_masters_degree','walk_score','bike_score']
key_col_comp = ['duns_number']
key_col_loc = ['atlas_location_uuid']

ind_city = 0

train_test_val_pairs = []
dat_comp_pds = []
dat_loc_pds = []


for ind_city in range(3):
    pdc = pd.read_csv(pjoin(datapath,cfile[ind_city]))
    pdl = pd.read_csv(pjoin(datapath,lfile))
    pdcl = pd.read_csv(pjoin(datapath,clfile[ind_city]))
    
    print('generating train_val_test csv')
    #train_test_val_pairs :[ duns_number, atlas_location_uuid, label, city, fold ]
    pair_dat = getPosNegdat(pdcl)
    tr,tt = splitdat(pair_dat,key_column=['duns_number','atlas_location_uuid'],right_colunm='label_tr',rate_tr=0.8)
    #training pair ==> pair format with positive only
    train_pos_pair = tr[tr['label']==1].groupby(['duns_number','atlas_location_uuid','label']).first().reset_index()[['duns_number','atlas_location_uuid','label']]
    #testing pair ==> pair format with positive and negative both
    testing_pair = tt.reset_index()[['duns_number','atlas_location_uuid','label']]
    
    train_pos_pair['fold'] = 0
    testing_pair['fold'] = 2

    train_test_val_pair = pd.concat([train_pos_pair,testing_pair])
    train_test_val_pair['city'] = ind_city
    train_test_val_pairs.append(train_test_val_pair)
    print(len(train_test_val_pair))
    print('train_val_test_location_company Done')
    
    
    #building feature
    proc_pdl = location_dat_process(pdl)
    col_list = list(proc_pdl['data'].columns)

    proc_pdll = proc_pdl['data'].merge(pdcl,how='inner',on=['atlas_location_uuid'],suffixes=['','_right'])
    proc_pdll = proc_pdll[proc_pdll['duns_number'].isnull()==False]
    proc_pdll = proc_pdll.groupby(['atlas_location_uuid']).first().reset_index()
    proc_pdll = proc_pdll[col_list]

    #company feature
    proc_pdc = comp_dat_process(pdc)
    print(len(proc_pdc))
    print(list(proc_pdc.columns),len(list(proc_pdc.columns)))
    
    continue
    print('start saving company and location feature...')

    XC_comp,XD_comp,Y_comp,c_comp_name,d_comp_name,y_comp_name = transpd2np_single(proc_pdc,cont_col_nameC,not_feat_col,id_col_name=key_col_comp)
    XC_loc,XD_loc,Y_loc,c_loc_name,d_loc_name,y_loc_name = transpd2np_single(proc_pdll,cont_col_nameL,not_feat_col,id_col_name=key_col_loc)

    C_comp,S_comp = get_para_normalize_dat(XC_comp)
    C_loc,S_loc = get_para_normalize_dat(XC_loc)
    XC_comp = apply_para_normalize_dat(XC_comp,C_comp,S_comp+1e-4)
    XC_loc = apply_para_normalize_dat(XC_loc,C_loc,S_loc+1e-4)
    
    X_comp = np.concatenate([Y_comp,XC_comp,XD_comp],axis=1)
    X_loc = np.concatenate([Y_loc,XC_loc,XD_loc],axis=1)

    dat_comp_pd = pd.DataFrame(data=X_comp,columns=y_comp_name+c_comp_name+d_comp_name)
    dat_loc_pd = pd.DataFrame(data=X_loc,columns=y_loc_name+c_loc_name+d_loc_name)
    
    print(dat_comp_pd.to_numpy().mean())
    print(dat_comp_pd.shape)
    dat_comp_pds.append(dat_comp_pd)
    dat_loc_pds.append(dat_loc_pd)
    print('Done')

return
print('Final merge...')
train_test_val_pair = pd.concat(train_test_val_pairs)
dat_comp_pd = pd.concat(dat_comp_pds)
dat_loc_pd = pd.concat(dat_loc_pds)

train_test_val_pair.to_csv('train_val_test_location_company_all.csv')
dat_comp_pd.to_csv('company_feat.csv')
dat_loc_pd.to_csv('location_feat.csv')
print('All Done')

generating train_val_test csv
6170
Neg dat num: 12340 ;Pos dat num: 6219
Train dat: 14847 Test dat: 3704
8690
train_val_test_location_company Done
doing one-hot...
extract continuous...
doing one-hot...
extract continuous...
specific feature
7538
['duns_number', 'emp_here', 'emp_total', 'sales_volume_us', 'square_footage', 'emp_here_range', 'major_industry_category_AGRICULTURE/FORESTRY/FISHING', 'major_industry_category_CONSTRUCTION', 'major_industry_category_FIN/INSURANCE/REAL ESTATE', 'major_industry_category_MANUFACTURING', 'major_industry_category_MINING', 'major_industry_category_MISC/PUBLIC ADMINISTRATION', 'major_industry_category_RETAIL TRADE', 'major_industry_category_SERVICES', 'major_industry_category_TRANS/COMMUNICATIONS/UTILITIES', 'major_industry_category_WHOLESALE TRADE', 'major_industry_category_nan', 'location_type_BRANCH', 'location_type_HEADQUARTERS', 'location_type_SINGLE', 'location_type_nan', 'primary_sic_2_digit_1', 'primary_sic_2_digit_13', 'primary_sic_2_digit_

SyntaxError: 'return' outside function (<ipython-input-70-b9a1bff95624>, line 98)

In [5]:
datapath = '/Users/yefeichen/Database/location_recommender_system/'
cfile = ['dnb_pa.csv','dnb_sf.csv','dnb_sj.csv']
lfile = 'location_scorecard_190912.csv'
clfile = ['PA.csv','SF.csv','SJ.csv']

not_feat_col = ['duns_number',
                 'atlas_location_uuid',
                 'longitude_loc',
                 'latitude_loc',
                 'label']
cont_col_nameC = ['emp_here','emp_total','sales_volume_us','square_footage','emp_here_range']
cont_col_nameL = ['score_predicted_eo','score_employer','num_emp_weworkcore','num_poi_weworkcore',
                 'pct_wwcore_employee','pct_wwcore_business','num_retail_stores','num_doctor_offices',
                 'num_eating_places','num_drinking_places','num_hotels','num_fitness_gyms',
                 'population_density','pct_female_population','median_age','income_per_capita',
                 'pct_masters_degree','walk_score','bike_score']
key_col_comp = ['duns_number']
key_col_loc = ['atlas_location_uuid']
pdc = pd.read_csv(pjoin(datapath,cfile[0]))


,duns_number,business_name,latitude,longitude,global_ultimate_duns_number,global_ultimate_business_name,emp_here,emp_here_code,emp_here_range,emp_total,...,primary_sic_4_digit,primary_sic_4_digit_v2,primary_sic_2_digit,primary_sic_2_digit_v2,naics_6_digit,naics_6_digit_v2,naics_4_digit,naics_4_digit_v2,naics_2_digit,naics_2_digit_v2
0,105571242,Scherb Consulting LLC,37.423371,-122.120151,105571242,Scherb Consulting LLC,1,US - modeled or Global -estimated or N/A when ...,1 - 4,1,...,8748,"BUSINESS CONSULTING, NEC",87,"ENGINEERING, ACCOUNTING, RESEARCH, AND MANAGEM...",541618,Other Management Consulting Services,5416,"Management, Scientific, and Technical Consulti...",54,"Professional, Scientific, and Technical Services"
1,555658061,"Engineering Software Concepts, Inc",37.444360,-122.161026,555658061,"Engineering Software Concepts, Inc",1,Actual,1 - 4,1,...,7373,COMPUTER INTEGRATED SYSTEMS DESIGN,73,BUSINESS SERVICES,541512,Computer Systems Design Services,5415,Computer Systems Design and Related Services,54,"Professional, Scientific, and Technical Services"
2,36779789,Opus Arcade,37.426430,-122.146146,36779789,Opus Arcade,10,Actual,10 - 14,10,...,7993,COIN-OPERATED AMUSEMENT DEVICES,79,AMUSEMENT AND RECREATION SERVICES,713120,Amusement Arcades,7131,Amusement Parks and Arcades,71,"Arts, Entertainment, and Recreation"
3,62393456,Loren Lam,37.436499,-122.128588,62393456,Loren Lam,1,Actual,1 - 4,1,...,8711,ENGINEERING SERVICES,87,"ENGINEERING, ACCOUNTING, RESEARCH, AND MANAGEM...",541330,Engineering Services,5413,"Architectural, Engineering, and Related Services",54,"Professional, Scientific, and Technical Services"
4,60656896,Sereno Group Real Estate,37.446594,-122.158823,67673174,Sereno Group Real Estate,3,US - modeled or Global -estimated or N/A when ...,1 - 4,3,...,6531,REAL ESTATE AGENTS AND MANAGERS,65,REAL ESTATE,531210,Offices of Real Estate Agents and Brokers,5312,Offices of Real Estate Agents and Brokers,53,Real Estate and Rental and Leasing


In [54]:
dat_loc_pd.to_numpy()[:,1:].mean(axis=0)

array([1.7612133250171304e-15, 1.1310171040766049e-15,
       -6.853005314868638e-17, -1.1418661890077953e-16,
       -8.200100027158119e-17, -1.8384678506313158e-16,
       -8.001200136752961e-18, 1.4465446574920607e-18,
       1.42846284927341e-17, 1.9889989040515833e-17,
       7.503950410740064e-17, 1.1165516575016843e-17,
       9.366376657261093e-17, 4.976113621772689e-16,
       -9.682808301087481e-16, 8.374137431262633e-17,
       -8.932413260013475e-17, -1.4601060136560487e-16,
       8.028322849080936e-17, 0.18729641693811075, 0.6172638436482085,
       0.1270358306188925, 0.06840390879478828], dtype=object)

In [12]:
#Is company key unique in all city set?
unique_comp = dat_comp_pd.groupby('duns_number').first()
assert(len(unique_comp)==len(dat_comp_pd))
print(len(unique_comp))

123764


In [13]:
#Is location key unique in all city set?
unique_loc = dat_loc_pd.groupby('atlas_location_uuid').first()
assert(len(unique_loc)==len(dat_loc_pd))
print(len(unique_loc))

2405


In [14]:
##Single training data generator
datapath = '/Users/yefeichen/Database/location_recommender_system/'
cfile = ['dnb_pa.csv','dnb_sf.csv','dnb_sj.csv']
lfile = 'location_scorecard_190912.csv'
clfile = ['PA.csv','SF.csv','SJ.csv']
lfile_app = ['PA_comp_loc_score.csv','SF_comp_loc_score.csv','SJ_comp_loc_score.csv']

ind_city = 0

pdc = pd.read_csv(pjoin(datapath,cfile[ind_city]))
pdl = pd.read_csv(pjoin(datapath,lfile))
pdcl = pd.read_csv(pjoin(datapath,clfile[ind_city]))



#building feature
proc_pdl = location_dat_process(pdl)

col_list = list(proc_pdl['data'].columns)

proc_pdll = proc_pdl['data'].merge(pdcl,how='inner',on=['atlas_location_uuid'],suffixes=['','_right'])
proc_pdll = proc_pdll[proc_pdll['duns_number'].isnull()==False]
proc_pdll = proc_pdll.groupby(['atlas_location_uuid']).first().reset_index()
proc_pdll = proc_pdll[col_list]

#company feature
proc_pdc = comp_dat_process(pdc)
print(len(proc_pdc))

print('generating train_val_test csv')
pair_dat = getPosNegdat(pdcl)
tr,tt = splitdat(pair_dat,key_column=['duns_number','atlas_location_uuid'],right_colunm='label_tr',rate_tr=0.8)
#training pair ==> pair format with positive only
train_pos_pair = tr[tr['label']==1].groupby(['duns_number','atlas_location_uuid','label']).first().reset_index()[['duns_number','atlas_location_uuid','label']]
#testing pair ==> pair format with positive and negative both
testing_pair = tt.reset_index()[['duns_number','atlas_location_uuid','label']]


train_pos_pair['fold'] = 0
testing_pair['fold'] = 2

train_test_val_pair = pd.concat([train_pos_pair,testing_pair])

print(len(train_test_val_pair))
train_test_val_pair.to_csv('train_val_test_location_company.csv')

print('save company and location feature processed.')
def transpd2np_single(featdat,cont_col_name:list,not_feat_col:list,id_col_name:list):
    XC = featdat.loc[:,cont_col_name].to_numpy()
    out_col = not_feat_col+cont_col_name
    dum_col_name = [col for col in list(featdat.columns) if col not in out_col]
    XD = featdat.loc[:,dum_col_name].to_numpy()
    Y = featdat[id_col_name].to_numpy()
    return XC,XD,Y,cont_col_name,dum_col_name,id_col_name

not_feat_col = ['duns_number',
                 'atlas_location_uuid',
                 'longitude_loc',
                 'latitude_loc',
                 'label']
cont_col_nameC = ['emp_here','emp_total','sales_volume_us','square_footage','emp_here_range']
cont_col_nameL = ['score_predicted_eo','score_employer','num_emp_weworkcore','num_poi_weworkcore',
                 'pct_wwcore_employee','pct_wwcore_business','num_retail_stores','num_doctor_offices',
                 'num_eating_places','num_drinking_places','num_hotels','num_fitness_gyms',
                 'population_density','pct_female_population','median_age','income_per_capita',
                 'pct_masters_degree','walk_score','bike_score']
key_col_comp = ['duns_number']
key_col_loc = ['atlas_location_uuid']

XC_comp,XD_comp,Y_comp,c_comp_name,d_comp_name,y_comp_name = transpd2np_single(proc_pdc,cont_col_nameC,not_feat_col,id_col_name=key_col_comp)
XC_loc,XD_loc,Y_loc,c_loc_name,d_loc_name,y_loc_name = transpd2np_single(proc_pdll,cont_col_nameL,not_feat_col,id_col_name=key_col_loc)

C_comp,S_comp = get_para_normalize_dat(XC_comp)
C_loc,S_loc = get_para_normalize_dat(XC_loc)
XC_comp = apply_para_normalize_dat(XC_comp,C_comp,S_comp)
XC_loc = apply_para_normalize_dat(XC_loc,C_loc,S_loc)

X_comp = np.concatenate([Y_comp,XC_comp,XD_comp],axis=1)
X_loc = np.concatenate([Y_loc,XC_loc,XD_loc],axis=1)

dat_comp_pd = pd.DataFrame(data=X_comp,columns=y_comp_name+c_comp_name+d_comp_name)
dat_loc_pd = pd.DataFrame(data=X_loc,columns=y_loc_name+c_loc_name+d_loc_name)

dat_comp_pd.to_csv('PA_company_feat.csv')
dat_loc_pd.to_csv('PA_loc_feat.csv')
print('Done')

AttributeError: 'DataFrame' object has no attribute 'shuffle'

In [85]:
dat = train_test_val_pair[train_test_val_pair['fold']==2]
# dat_comp_pd
# dat_loc_pd

posN = 100
negN = 200

dat1 = dat.sample(n=posN).reset_index(drop=True)
dat2 = dat1.sample(frac=1).reset_index(drop=True)

twin_dat = pd.merge(dat1,dat2,on='city',how='left',suffixes=['_left','_right'])
twin_dat = twin_dat[twin_dat['atlas_location_uuid_left'] != twin_dat['atlas_location_uuid_right']]
# print(len(twin_dat))
# twin_dat.head()
neg_dat = twin_dat[['duns_number_left','atlas_location_uuid_right']].sample(n=negN).reset_index(drop=True)
neg_dat = neg_dat.rename(columns={'duns_number_left':'duns_number','atlas_location_uuid_right':'atlas_location_uuid'})
neg_dat['label'] = 0
# neg_dat.head()
# len(neg_dat)
pos_dat = dat1[['duns_number','atlas_location_uuid','label']]
res_dat = pd.concat([pos_dat,neg_dat],axis=0)
res_dat = res_dat.sample(frac=1).reset_index(drop=True)

# concate training pair with location/company feature
F_res_dat = pd.merge(res_dat, dat_comp_pd, on='duns_number', how='left')
list_col = list(F_res_dat.columns)
list_col = [col for col in list_col if col not in ['duns_number', 'atlas_location_uuid', 'label']]
FeatComp = F_res_dat[list_col].to_numpy()

F_res_dat = pd.merge(res_dat, dat_loc_pd, on='atlas_location_uuid', how='left')
list_col = list(F_res_dat.columns)
list_col = [col for col in list_col if col not in ['duns_number', 'atlas_location_uuid', 'label']]
FeatLoc = F_res_dat[list_col].to_numpy()
#[B,C,W,H]
#[B,Len]

In [86]:
print(list_col)

['score_predicted_eo', 'score_employer', 'num_emp_weworkcore', 'num_poi_weworkcore', 'pct_wwcore_employee', 'pct_wwcore_business', 'num_retail_stores', 'num_doctor_offices', 'num_eating_places', 'num_drinking_places', 'num_hotels', 'num_fitness_gyms', 'population_density', 'pct_female_population', 'median_age', 'income_per_capita', 'pct_masters_degree', 'walk_score', 'bike_score', 'building_class_A', 'building_class_B', 'building_class_C', 'building_class_nan']


In [102]:
dat = train_test_val_pair[train_test_val_pair['fold']==2]
dat = dat[['duns_number','atlas_location_uuid','label']]
F_res_dat = pd.merge(dat,dat_comp_pd,on='duns_number',how='left')
F_res_dat = pd.merge(F_res_dat,dat_loc_pd,on='atlas_location_uuid',how='left')
list_col = list(F_res_dat.columns)
list_col = [col for col in list_col if col not in ['duns_number','atlas_location_uuid','label'] ]
Feat = F_res_dat[list_col].to_numpy()
Label = F_res_dat[['label']].to_numpy()

In [66]:
F_res_dat[list_col].head()

,emp_here,emp_here_range,emp_total,location_type_BRANCH,location_type_HEADQUARTERS,location_type_SINGLE,location_type_nan,major_industry_category_AGRICULTURE/FORESTRY/FISHING,major_industry_category_CONSTRUCTION,major_industry_category_FIN/INSURANCE/REAL ESTATE,...,pct_female_population,median_age,income_per_capita,pct_masters_degree,walk_score,bike_score,building_class_A,building_class_B,building_class_C,building_class_nan
0,-0.102383,-0.235950,-0.033227,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.904084,1.02456,0.324421,-0.0875472,0.57438,-0.409338,1,0,0,0
1,-0.102383,-0.235950,-0.033227,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.904084,1.02456,0.324421,-0.0875472,0.57438,-0.409338,1,0,0,0
2,-0.084579,-0.235950,-0.031005,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.418255,-1.25815,-0.765051,0.382038,0.614712,0.656537,0,1,0,0
3,-0.093481,-0.235950,-0.032116,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0783954,0.187155,-0.232348,0.547229,0.493717,0.656537,0,0,1,0
4,0.022244,0.198956,-0.017670,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,-0.601254,-1.03462,-0.25927,0.461617,-0.0709239,-0.118645,0,1,0,0


In [68]:
F_res_dat[list_col].head()

,emp_here,emp_here_range,emp_total,location_type_BRANCH,location_type_HEADQUARTERS,location_type_SINGLE,location_type_nan,major_industry_category_AGRICULTURE/FORESTRY/FISHING,major_industry_category_CONSTRUCTION,major_industry_category_FIN/INSURANCE/REAL ESTATE,...,pct_female_population,median_age,income_per_capita,pct_masters_degree,walk_score,bike_score,building_class_A,building_class_B,building_class_C,building_class_nan
0,-0.123628,-0.209948,-0.019855,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.613826,-1.27097,-1.61284,-1.28336,0.292841,-0.124552,1,0,0,0
1,-0.064590,-0.083606,-0.019283,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.848538,0.143898,0.0728309,0.971867,-1.04857,-0.665242,0,1,0,0
2,-0.087412,-0.222866,-0.020740,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.727245,-0.477956,-0.547845,0.541462,0.331178,-2.16022,0,1,0,0
3,-0.110233,-0.222866,-0.022196,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-1.39076,1.3513,1.27981,1.68832,-0.220721,-1.96089,0,1,0,0
4,-0.076001,-0.222866,-0.020011,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.06182,0.219803,-1.1685,-1.28389,0.0552282,-3.6552,0,0,0,1


In [69]:
pdtmp = pd.read_csv(pjoin(datapath,'SF_comp_loc_score.csv'))

In [70]:
pdtmp.head()

,duns_number,atlas_location_uuid,sim_score
0,1014492.0,00114df7-7381-b92a-7850-7272002508a2,2.041952
1,1014492.0,002f21b2-9bbe-72eb-ca46-09f08174abfd,1.955841
2,1014492.0,00a03ec6-533c-a7f5-c644-d198d815329c,1.414592
3,1014492.0,00a7b8f6-dc44-0f57-9951-5d58dafdb5be,1.808272
4,1014492.0,00cbf276-2187-a65e-e084-69b2bf07e898,1.808272


In [71]:
pdtmp.shape

(81289110, 3)

In [112]:
res_dat = dat[['duns_number','atlas_location_uuid','label']].iloc[0]



duns_number    2.0359e+06
Name: 0, dtype: object

In [103]:
dat = dat.reset_index()
# res_dat = dat.loc[0,['duns_number','atlas_location_uuid','label']]

In [96]:
res_dat

,duns_number,atlas_location_uuid,label
0,2035902.0,77c493ec-1424-6d74-8db3-ee8fce0092db,1
1,2114703.0,77c493ec-1424-6d74-8db3-ee8fce0092db,1


In [120]:
res_dat = dat.loc[0:0,['duns_number','atlas_location_uuid','label']]
res_dat

,duns_number,atlas_location_uuid,label
0,2035902.0,77c493ec-1424-6d74-8db3-ee8fce0092db,1


In [4]:

# dat_comp_pd = dat_comp_pd.fillna(0)
K = dat_comp_pd.to_numpy()

In [5]:
K.mean(axis=0)

array([ 2.19044741e+08, -3.44466599e-18, -7.75049848e-18,  9.25753985e-19,
        8.20109240e-02,  3.26508516e-02,  8.85338224e-01,  0.00000000e+00,
        1.24753563e-02,  6.27888562e-02,  1.01176433e-01,  3.69251155e-02,
        4.60553958e-04,  6.77095117e-03,  1.41648622e-01,  5.67192398e-01,
        3.49617013e-02,  3.56000129e-02,  0.00000000e+00,  1.81797615e-03,
                   nan,             nan,  2.74716396e-04,             nan,
        2.96612908e-02,  1.27662325e-03,  3.18509421e-02,  7.43350247e-04,
        2.87644226e-03,             nan,  4.68633852e-04,  1.34126240e-03,
        1.21198410e-03,  6.54471413e-04,             nan,  6.67399244e-03,
        2.22197085e-03,             nan,  4.36314276e-04,             nan,
        7.43350247e-04,  2.74716396e-04,  1.72909731e-03,  4.37122265e-03,
        6.05992049e-03,  5.57512685e-04,  3.29659675e-03,  3.44203484e-03,
                   nan,  3.85410943e-03,  6.66591254e-03,  3.95914806e-04,
        5.65592579e-04,  

In [139]:
dat_loc_pd = dat_loc_pd.fillna(0)
K = dat_loc_pd.to_numpy()
K[:,1:].mean(axis=0)

array([5.134637229142827e-16, 1.2136284123657117e-16,
       2.6543793728044285e-17, -9.414968227538489e-17,
       -2.2139832125165285e-16, 3.833385445524865e-16,
       -4.151218523468491e-17, -3.2002582612116e-17,
       4.907139605723665e-17, 7.372250188467255e-17,
       -2.3820169675949306e-17, 1.2267849014309163e-17,
       1.372429543538707e-16, 1.2496356455967977e-16,
       4.481054012489146e-16, 1.2867969568160598e-16,
       8.415536689778213e-17, -9.24301060730029e-17,
       -2.5001945535841364e-16, 0.15051975051975053, 0.5455301455301456,
       0.22952182952182953, 0.07442827442827443], dtype=object)

In [138]:
dat_loc_pd

,atlas_location_uuid,score_predicted_eo,score_employer,num_emp_weworkcore,num_poi_weworkcore,pct_wwcore_employee,pct_wwcore_business,num_retail_stores,num_doctor_offices,num_eating_places,...,pct_female_population,median_age,income_per_capita,pct_masters_degree,walk_score,bike_score,building_class_A,building_class_B,building_class_C,building_class_nan
0,003fd16c-0dee-24a1-1adf-0b47d6120f3d,-0.108268,-0.812375,-1.131663,-0.484551,-1.371326,-0.180111,-0.650659,-0.536782,-0.264323,...,0.343029,-1.316975,-0.877594,0.392834,0.655043,0.753435,0,1,0,0
1,0060ef47-b121-60b9-1983-02ebdb144dcf,0.911005,0.545407,-0.833037,-0.454106,-0.768515,-0.187081,-0.650659,-0.536782,-0.294269,...,0.402471,-1.357497,-0.850475,0.535219,0.655043,0.753435,0,1,0,0
2,0084ae4b-c24c-0795-d1e6-a4f58444d39b,-0.691620,0.545407,-1.746886,-1.425300,0.114733,0.956742,-1.150183,-1.252198,-1.581941,...,0.090320,1.584967,1.208156,0.956204,-2.289157,-2.056599,1,0,0,0
3,00a68dc6-f955-c581-9a54-64404bb0ef6a,-1.062121,0.647241,0.333720,-1.449656,-0.129408,0.498155,-1.177934,-0.785622,-1.671778,...,-0.141400,1.067323,0.089986,0.455658,-2.087499,-1.959702,0,1,0,0
4,01109439-6b76-0fc9-2d96-cfdb79708a80,0.788091,0.443574,-1.041811,-0.877291,-0.138564,0.311138,-0.733913,-0.956700,-0.803349,...,0.497937,-1.954879,-0.994768,0.543390,0.050071,-0.021747,0,1,0,0
5,018dd1e0-7a2d-e4a0-8e66-12341bf2323e,0.081267,-0.744486,0.609355,0.367908,0.795177,-0.013035,0.292887,0.987363,0.723890,...,0.482602,-0.279073,0.506074,1.146066,0.695375,0.753435,0,1,0,0
6,019876c4-3e88-93db-b2d1-cbbac77eb59b,0.102025,-0.778431,1.075265,0.876339,0.763039,0.190504,1.680454,1.220651,1.113186,...,-1.141543,0.061578,0.995053,-0.629779,0.776038,0.753435,1,0,0,0
7,0282ea12-9cd8-099b-654d-a64c88c6347f,-0.113548,-0.778431,0.968500,0.812405,0.926325,0.125055,0.681406,1.049573,0.933511,...,-0.007752,-0.160991,0.353538,1.012560,0.816369,0.753435,0,1,0,0
8,0283c67f-28df-2602-2c62-f493ea8f3890,0.108423,2.174746,1.022147,-1.306565,0.190002,1.407735,-1.150183,-1.221093,-1.641832,...,-0.141400,1.067323,0.089986,0.455658,-2.168162,-2.056599,0,0,1,0
9,03362430-41b8-2fe8-f237-23aa1abbca51,-0.054662,-0.506874,-1.119242,-0.444972,-1.092092,-0.200154,-0.595156,-0.645650,-0.144540,...,0.681546,-1.052271,-0.371148,0.344253,0.574380,0.559639,0,1,0,0
